## Graph Attention Network (GAT)

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [16]:
class GATLayer(nn.Module):
    def __init__(self, g, in_dim, out_dim):
        super(GATLayer, self).__init__()
        self.g = g
        self.fc = nn.Linear(in_dim, out_dim, bias=False)
        self.attn_fc = nn.Linear(2*out_dim, 1, bias=False)
    
    def edge_attention(self, edges):
        # dim=1 denotes concatenation along 1th dimension (0-based)
        z2 = torch.cat([edges.src['z'], edges.dst['z']], dim=1)
        a = self.attn_fc(z2)
        return {'e' : F.leaky_relu(a)}
    
    def message_func(self, edges):
        return {'z' : edges.src['z'], 'e' : edges.data['e']}
        
    def reduce_func(self, nodes):
        alpha = F.softmax(nodes.mailbox['e'], dim=1)
        h = torch.sum(alpha * nodes.mailbox['z'], dim=1)
        return {'h' : h}
    
    def forward(self, h):
        z = self.fc(h)
        self.g.ndata['z'] = z
        self.g.apply_edges(self.edge_attention)
        self.g.update_all(self.message_func, self.reduce_func)
        
        return self.g.ndata.pop('h')

In [17]:
class MultiHeadGATLayer(nn.Module):
    def __init__(self, g, in_dim, out_dim, num_heads, merge='cat'):
        super(MultiHeadGATLayer, self).__init__()
        self.heads = nn.ModuleList()
        for i in range(num_heads):
            self.heads.append(GATLayer(g, in_dim, out_dim))
            self.merge = merge
    
    def forward(self, h):
        head_outs = [attn_head(h) for attn_head in self.heads]
        if self.merge == 'cat':
            return torch.cat(head_outs, dim=1)
        else:
            return torch.mean(torch.stack(head_outs))

In [18]:
class GAT(nn.Module):
    def __init__(self, g, in_dim, hidden_dim, out_dim, num_heads):
        super(GAT, self).__init__()
        self.layer1 = MultiHeadGATLayer(g, in_dim, hidden_dim, num_heads)
        self.layer2 = MultiHeadGATLayer(g, hidden_dim*num_heads, out_dim, 1)
    
    def forward(self, h):
        h = self.layer1(h)
        h = F.elu(h)
        h = self.layer2(h)
        return h

In [19]:
from dgl import DGLGraph
from dgl.data import citation_graph as citegrh

def load_cora_data():
    data = citegrh.load_cora()
    features = torch.FloatTensor(data.features)
    labels = torch.LongTensor(data.labels)
    mask = torch.ByteTensor(data.train_mask)
    g = DGLGraph(data.graph)
    return g, features, labels, mask

In [20]:
import time
import numpy as np
g, features, labels, mask = load_cora_data()

# create the model
net = GAT(g, 
          in_dim=features.size()[1], 
          hidden_dim=8, 
          out_dim=7, 
          num_heads=8)
print(net)

# create optimizer
optimizer = torch.optim.Adam(net.parameters(), lr=1e-3)

# main loop
dur = []
for epoch in range(30):
    if epoch >=3:
        t0 = time.time()
        
    logits = net(features)
    logp = F.log_softmax(logits, 1)
    loss = F.nll_loss(logp[mask], labels[mask])
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if epoch >=3:
        dur.append(time.time() - t0)
    
    print("Epoch {:05d} | Loss {:.4f} | Time(s) {:.4f}".format(
            epoch, loss.item(), np.mean(dur)))

GAT(
  (layer1): MultiHeadGATLayer(
    (heads): ModuleList(
      (0): GATLayer(
        (fc): Linear(in_features=1433, out_features=8, bias=False)
        (attn_fc): Linear(in_features=16, out_features=1, bias=False)
      )
      (1): GATLayer(
        (fc): Linear(in_features=1433, out_features=8, bias=False)
        (attn_fc): Linear(in_features=16, out_features=1, bias=False)
      )
      (2): GATLayer(
        (fc): Linear(in_features=1433, out_features=8, bias=False)
        (attn_fc): Linear(in_features=16, out_features=1, bias=False)
      )
      (3): GATLayer(
        (fc): Linear(in_features=1433, out_features=8, bias=False)
        (attn_fc): Linear(in_features=16, out_features=1, bias=False)
      )
      (4): GATLayer(
        (fc): Linear(in_features=1433, out_features=8, bias=False)
        (attn_fc): Linear(in_features=16, out_features=1, bias=False)
      )
      (5): GATLayer(
        (fc): Linear(in_features=1433, out_features=8, bias=False)
        (attn_fc): Li

/home/nikhil.bansal/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/nikhil.bansal/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Epoch 00000 | Loss 1.9461 | Time(s) nan
Epoch 00001 | Loss 1.9445 | Time(s) nan
Epoch 00002 | Loss 1.9429 | Time(s) nan
Epoch 00003 | Loss 1.9414 | Time(s) 0.9322
Epoch 00004 | Loss 1.9397 | Time(s) 0.9353
Epoch 00005 | Loss 1.9381 | Time(s) 0.9365
Epoch 00006 | Loss 1.9364 | Time(s) 0.9546
Epoch 00007 | Loss 1.9346 | Time(s) 0.9987
Epoch 00008 | Loss 1.9328 | Time(s) 0.9934
Epoch 00009 | Loss 1.9310 | Time(s) 0.9986
Epoch 00010 | Loss 1.9291 | Time(s) 1.0102
Epoch 00011 | Loss 1.9272 | Time(s) 1.0133
Epoch 00012 | Loss 1.9252 | Time(s) 1.0245
Epoch 00013 | Loss 1.9231 | Time(s) 1.0355
Epoch 00014 | Loss 1.9210 | Time(s) 1.0415
Epoch 00015 | Loss 1.9188 | Time(s) 1.0442
Epoch 00016 | Loss 1.9165 | Time(s) 1.0415
Epoch 00017 | Loss 1.9142 | Time(s) 1.0346
Epoch 00018 | Loss 1.9118 | Time(s) 1.0325
Epoch 00019 | Loss 1.9094 | Time(s) 1.0334
Epoch 00020 | Loss 1.9069 | Time(s) 1.0297
Epoch 00021 | Loss 1.9043 | Time(s) 1.0287
Epoch 00022 | Loss 1.9016 | Time(s) 1.0366
Epoch 00023 | Loss 1